# `codex-cli`でJira ↔ GitHubを自動化

## このクックブックの目的

このクックブックは、JiraとGitHub間のワークフローを自動化するための実践的なステップバイステップのアプローチを提供します。Jiraイシューにラベルを付けることで、**GitHubプルリクエスト**を作成し、両システムを最新の状態に保ち、コードレビューを効率化するエンドツーエンドのプロセスを、最小限の手動作業でトリガーできます。この自動化は、GitHub Action内で実行される[`codex-cli`](https://github.com/openai/openai-codex)エージェントによって実現されています。

<img src="../../images/codex_action.png" alt="完全なデータフロー図" width="500"/>

フローは以下の通りです：

1. Jiraチケットにラベルを付ける
2. Jira AutomationがGitHub Actionを呼び出す
3. アクションが`codex-cli`を起動して変更を実装する
4. PRが開かれる
5. Jiraが遷移・注釈される - 整然としたゼロクリックループを作成します。これには、チケットのステータス変更、PRリンクの追加、チケットへの更新コメントが含まれます。

## 前提条件

* Jira: プロジェクト管理者権限 + 自動化ルールを作成する権限
* GitHub: 書き込みアクセス権、リポジトリシークレットを追加する権限、保護された`main`ブランチ
* リポジトリシークレットとして配置されたAPIキーとシークレット：
  * `OPENAI_API_KEY` – `codex-cli`用のOpenAIキー
  * `JIRA_BASE_URL`、`JIRA_EMAIL`、`JIRA_API_TOKEN` – アクションからのREST呼び出し用
* アドホックテスト用にローカルにインストールされた`codex-cli`（`pnpm add -g @openai/codex`）
* `.github/workflows/`フォルダを含むリポジトリ

## Jira自動化ルールの作成

<img src="../../images/jira_rule.png" alt="Automation Rule" width="500"/>

このルールの最初のステップでは、課題のラベルの変更を監視します。これにより、ラベルが追加または変更された場合にのみ自動化がトリガーされ、課題のすべての更新を処理する必要がなくなります。

次に、更新されたラベルに特定のキーワードが含まれているかどうかを確認します。この例では`aswe`を使用しています。これはフィルターとして機能し、自動化のために明示的にタグ付けされた課題のみが処理され、関連のない更新による不要なノイズを回避できます。

条件が満たされた場合、GitHubの`workflow_dispatch`エンドポイントに`POST`リクエストを送信します。これにより、関連する課題のコンテキストを含むGitHub Actionsワークフローが開始されます。課題キー、概要、および説明のクリーンアップされたバージョンを渡します。引用符と改行をエスケープして、ペイロードがYAML/JSONで正しく解析されるようにします。JIRAには[追加フィールド](https://support.atlassian.com/cloud-automation/docs/jira-smart-values-issues/)が変数として利用可能で、実行中にcodexエージェントにより多くのコンテキストを提供できます。

このセットアップにより、チームはどのJira課題が自動化をトリガーするかを厳密に制御でき、GitHubが構造化されたクリーンなメタデータを受信して処理できることが保証されます。また、複数のラベルを設定し、それぞれが異なるGitHub Actionをトリガーするようにすることも可能です。例えば、あるラベルは迅速なバグ修正ワークフローを開始し、別のラベルはコードのリファクタリングやAPIスタブの生成作業を開始するかもしれません。

## GitHub Actionの追加

GitHub Actionsを使用すると、YAMLファイルでワークフローを定義することで、GitHubリポジトリ内でワークフローを自動化できます。これらのワークフローは、実行する一連のジョブとステップを指定します。手動またはPOSTリクエストによってトリガーされると、GitHubは自動的に必要な環境をプロビジョニングし、定義されたワークフローステップを実行します。

JIRAからの`POST`リクエストを処理するために、リポジトリの`.github/workflows/`ディレクトリに以下のようなYAMLでGitHub Actionを作成します：

```yaml
name: Codex 自動PR
on:
  workflow_dispatch:
    inputs:
      issue_key:
        description: 'JIRA課題キー（例：PROJ-123）'
        required: true
      issue_summary:
        description: '課題の簡潔な要約'
        required: true
      issue_description:
        description: '詳細な課題説明'
        required: true

permissions:
  contents: write           # アクションがコードをプッシュしてPRを開くことを許可
  pull-requests: write      # アクションがPRを作成・更新することを許可

jobs:
  codex_auto_pr:
    runs-on: ubuntu-latest

    steps:
    # 0 – リポジトリをチェックアウト
    - uses: actions/checkout@v4
      with:
        fetch-depth: 0       # 完全な履歴 → 必要に応じてCodexがテストやgit blameを実行できるように

    # 1 – Node.jsとCodexをセットアップ
    - uses: actions/setup-node@v4
      with:
        node-version: 22
    - run: pnpm add -g @openai/codex

    # 2 – 入力をエクスポート/クリーンアップ（$GITHUB_ENV経由で利用可能）
    - id: vars
      run: |
        echo "ISSUE_KEY=${{ github.event.inputs.issue_key }}"        >> $GITHUB_ENV
        echo "TITLE=${{ github.event.inputs.issue_summary }}"        >> $GITHUB_ENV
        echo "RAW_DESC=${{ github.event.inputs.issue_description }}" >> $GITHUB_ENV
        DESC_CLEANED=$(echo "${{ github.event.inputs.issue_description }}" | tr '\n' ' ' | sed 's/"/'\''/g')
        echo "DESC=$DESC_CLEANED"                                    >> $GITHUB_ENV
        echo "BRANCH=codex/${{ github.event.inputs.issue_key }}"     >> $GITHUB_ENV

    # 3 – Jira課題を「進行中」に遷移
    - name: Jira – 進行中に遷移
      env:
        ISSUE_KEY:      ${{ env.ISSUE_KEY }}
        JIRA_BASE_URL:  ${{ secrets.JIRA_BASE_URL }}
        JIRA_EMAIL:     ${{ secrets.JIRA_EMAIL }}
        JIRA_API_TOKEN: ${{ secrets.JIRA_API_TOKEN }}
      run: |
        curl -sS -X POST \
          --url   "$JIRA_BASE_URL/rest/api/3/issue/$ISSUE_KEY/transitions" \
          --user  "$JIRA_EMAIL:$JIRA_API_TOKEN" \
          --header 'Content-Type: application/json' \
          --data  '{"transition":{"id":"21"}}'
          # 21はチケットステータスを進行中に変更するための遷移IDです。詳細はこちら: https://developer.atlassian.com/cloud/jira/platform/rest/v3/api-group-issues/#api-rest-api-3-issue-issueidorkey-transitions-get

    # 4 – CIコミット用のGit作成者を設定
    - run: |
        git config user.email "github-actions[bot]@users.noreply.github.com"
        git config user.name  "github-actions[bot]"

    # 5 – Codexに実装とコミットを実行させる（まだプッシュしない）
    - name: Codex実装とコミット
      env:
        OPENAI_API_KEY:  ${{ secrets.OPENAI_API_KEY }}
        CODEX_QUIET_MODE: "1"          # 冗長なログを抑制
      run: |
        set -e
        codex --approval-mode full-auto --no-terminal --quiet \
              "Implement JIRA ticket $ISSUE_KEY: $TITLE. $DESC"

        git add -A
        git commit -m "feat($ISSUE_KEY): $TITLE"

    # 6 – PRを一度に開く（そしてプッシュ）
    - id: cpr
      uses: peter-evans/create-pull-request@v6
      with:
        token:  ${{ secrets.GITHUB_TOKEN }}
        base:   main
        branch: ${{ env.BRANCH }}
        title:  "${{ env.TITLE }} (${{ env.ISSUE_KEY }})"
        body: |
          JIRA **${{ env.ISSUE_KEY }}** 用にCodexによって自動生成されました。
          ---
          ${{ env.DESC }}

    # 7 – Jiraを「レビュー中」に遷移してPRリンクを追加
    - name: Jira – レビュー中に遷移してPRリンクをコメント
      env:
        ISSUE_KEY:      ${{ env.ISSUE_KEY }}
        JIRA_BASE_URL:  ${{ secrets.JIRA_BASE_URL }}
        JIRA_EMAIL:     ${{ secrets.JIRA_EMAIL }}
        JIRA_API_TOKEN: ${{ secrets.JIRA_API_TOKEN }}
        PR_URL:         ${{ steps.cpr.outputs.pull-request-url }}
      run: |
        # ステータス遷移
        curl -sS -X POST \
          --url   "$JIRA_BASE_URL/rest/api/3/issue/$ISSUE_KEY/transitions" \
          --user  "$JIRA_EMAIL:$JIRA_API_TOKEN" \
          --header 'Content-Type: application/json' \
          --data  '{"transition":{"id":"31"}}'
          # 31はチケットステータスをレビュー中に変更するための遷移IDです。詳細はこちら: https://developer.atlassian.com/cloud/jira/platform/rest/v3/api-group-issues/#api-rest-api-3-issue-issueidorkey-transitions-get

        # PRリンクをコメント
        curl -sS -X POST \
          --url   "$JIRA_BASE_URL/rest/api/3/issue/$ISSUE_KEY/comment" \
          --user  "$JIRA_EMAIL:$JIRA_API_TOKEN" \
          --header 'Content-Type: application/json' \
          --data  "{\"body\":{\"type\":\"doc\",\"version\":1,\"content\":[{\"type\":\"paragraph\",\"content\":[{\"type\":\"text\",\"text\":\"PR created: $PR_URL\"}]}]}}"
```

## ワークフローの主要ステップ

1. **Codex実装とコミット** (ステップ5)
   - OpenAI APIを使用してJIRAチケットの要件を実装
   - ターミナルでの対話なしにcodex CLIをフル自動モードで実行
   - 標準化されたコミットメッセージですべての変更をコミット

2. **プルリクエストの作成** (ステップ6) 
   - peter-evans/create-pull-requestアクションを使用
   - mainブランチに対してPRを作成
   - JIRAチケット情報からPRのタイトルと説明を設定
   - 後で使用するためにPR URLを返す

3. **JIRAの更新** (ステップ7)
   - JIRA APIを通じてチケットを「レビュー中」ステータスに移行
   - JIRAチケットにPR URLを含むコメントを投稿
   - curlコマンドを使用してJIRA REST APIと連携

## Issue にラベルを付ける

バグ/機能チケットに特別な `aswe` ラベルを付けてください：

1. **作成時** – *Create* をクリックする前に「Labels」フィールドで追加する
2. **既存のissue** – ラベルエリアにホバー → 鉛筆アイコンをクリック → `aswe` と入力

<img src="../../images/add_label.png" alt="Adding a label" width="500"/>

## エンドツーエンドフロー

1. Jiraラベルが追加される → 自動化がトリガーされる
2. `workflow_dispatch`が実行される；アクションがGitHub上で開始される
3. `codex-cli`がコードベースを編集してコミットする
4. 生成されたブランチでPRが開かれる
5. Jiraが**In Review**に移動され、PR URLを含むコメントが投稿される
6. 通常のブランチ保護設定に従ってレビュアーに通知される

<img src="../../images/jira_comment.png" alt="PR リンク付きJiraコメント" width="300"/>
<img src="../../images/jira_status_change.png" alt="Jiraステータスの In Review への変更" width="300"/>

## PRのレビューとマージ

JIRAチケットに投稿されたPRリンクを開いて、すべてが正常に見えるかを確認し、その後マージすることができます。ブランチ保護とSmart Commitsの統合が有効になっている場合、プルリクエストがマージされると、Jiraチケットは自動的にクローズされます。

## まとめ

この自動化により、JiraとGitHub間のシームレスな統合を作成することで、開発ワークフローが効率化されます：

* **自動ステータス追跡** - チケットは手動更新なしでワークフローを進行します
* **開発者体験の向上** - 定型コードを書く代わりに、コード品質のレビューに集中できます
* **引き継ぎ摩擦の軽減** - チケットにラベルが付けられるとすぐにPRがレビュー準備完了状態になります

`codex-cli`ツールは、反復的なプログラミングタスクを自動化する強力なAIコーディングアシスタントです。詳細については[こちら](https://github.com/openai/codex/)で確認できます。